<a href="https://colab.research.google.com/github/AlekseiDiazPUCP/GCP/blob/main/CloudFunctionsRepository.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Cloud Functions
<p>This notebook is a collection of different functions I have created. The pourpose is to provide a repository to easily review them</p> 222


##**Cloud Storage**

### Copy a file from one bucket to another


**requirements.txt**
<p> google-cloud-storage</p>

In [ ]:
from google.cloud import storage
def copy_blob(bucket_name, blob_name, destination_bucket_name, destination_blob_name):
    """Copies a blob from one bucket to another with a new name."""
    # bucket_name = "your-bucket-name"
    # blob_name = "your-object-name"
    # destination_bucket_name = "destination-bucket-name"
    # destination_blob_name = "destination-object-name"

    storage_client = storage.Client()

    source_bucket = storage_client.bucket(bucket_name)
    source_blob = source_bucket.blob(blob_name)
    destination_bucket = storage_client.bucket(destination_bucket_name)

    blob_copy = source_bucket.copy_blob(
        source_blob, destination_bucket, destination_blob_name
    )
    print(
        "Blob {} in bucket {} copied to blob {} in bucket {}.".format(
            source_blob.name,
            source_bucket.name,
            blob_copy.name,
            destination_bucket.name,
        )
    )

### Delete a file from a bucket

**requirements.txt**
<p> google-cloud-storage</p>

In [ ]:
from google.cloud import storage
def delete_blob(bucket_name, blob_name):
    """Deletes a blob from the bucket."""
    # bucket_name = "your-bucket-name"
    # blob_name = "your-object-name"

    storage_client = storage.Client()

    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.delete()

    print("Blob {} deleted.".format(blob_name))


## **BigQuery**

### Run Scheduled Queries on demmand

**requirements.txt**
<p> google-cloud-bigquery-datatransfer==1</p>

In [ ]:
from google.cloud import bigquery_datatransfer_v1
import time

def runScheduledQuery(resourceName):
  """runs a scheduled query on demmand"""
  #resourceName: Can be found in the scheduled query configuration under the name of Resource Name
  client = bigquery_datatransfer_v1.DataTransferServiceClient()
  parent = resourceName
  print(f'paren is {parent}')
  start_time = bigquery_datatransfer_v1.types.Timestamp(seconds=int(time.time() + 10))
  response = client.start_manual_transfer_runs(parent, requested_run_time=start_time)
  return "query has been triggered"


### Run Custom Queries

**requirements.txt**
<p> google-cloud-bigquery</p>

In [ ]:
from google.cloud import bigquery
import time

def runQuery(query):
    # Construct a BigQuery client object.
    client = bigquery.Client()

    query_job = client.query(query)  # Make an API request.

    status = query_job.state

    while status == 'RUNNING':
      print('the query is still running')
      time.sleep(5)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      status = query_job.state


### **Loading a Data Frame**

**requirements.txt**
<p> google-cloud-bigquery</p>
<p> pandas </p>

In [ ]:
import pandas as pd
from google.cloud import bigquery

def loadDataFrame(job_config,df,table_id):
  """Triggered from a message on a Cloud Pub/Sub topic.
    Args:
         job_config (variable): The Job configuration. There is an example below.
         df (dataframe): df is a pandas dataframe to be loaded into a BigQuery table
         table_id (string) : It is the table id which df will be loaded. projec_id.dataset_id.table_id 
         context (google.cloud.functions.Context): Metadata for the event.
    """

  """
  job_config = bigquery.LoadJobConfig(
        schema=[
          bigquery.SchemaField("flow_name", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("communication_start_date_time", bigquery.enums.SqlTypeNames.STRING),
          bigquery.SchemaField("click", bigquery.enums.SqlTypeNames.INTEGER),
          bigquery.SchemaField("send_at_date", bigquery.enums.SqlTypeNames.DATE)
          ]
        )
  """
  job = client.load_table_from_dataframe(df, table_id,job_config=job_config) 
  
  return "200" 



### **Loading a JSON**

**requirements.txt**
<p> google-cloud-bigquery</p>

In [ ]:
from google.cloud import bigquery

#data must be from a get_json for example

def loadJsonToBQ(json, projectId, datasetId, tableId):
    
  client = bigquery.Client()

  data = [data]

  job = client.load_table_from_json(data,table_id)

  return '200'        





  


## **Pub/Sub**

### **Publish in  Topic**

**requirements.txt**
<p> google-cloud-pubsub</p>

In [ ]:
from google.cloud import pubsub_v1
import json


def publish(bucket_name,file_name,topic_name):
  # Publish to a topic.

  publisher = pubsub_v1.PublisherClient()
  
  # References an existing topic
  message_json = json.dumps({
      'data':  {'bucket_name': bucket_name,'file_name':file_name},
    })
  message_bytes = str(message_json).encode('utf-8')

  # Publishes a message
  try:
    publish_future = publisher.publish(topic_path, data=message_bytes)
    publish_future.result()  # Verify the publish succeeded
    return 'Message published.'
  except Exception as e:
    print(e)
    return (e, 500)

### **Read from a Topic**

**requirements.txt**
<p> google-cloud-pubsub</p>

In [ ]:
from google.cloud import pubsub_v1
import json


def parseMessage(event, context):
    """Triggered from a message on a Cloud Pub/Sub topic.
    Args:
         event (dict): Event payload.
         context (google.cloud.functions.Context): Metadata for the event.
    """
    pubsub_message = base64.b64decode(event['data']).decode('utf-8')
    
    #Make the message data accesible form key values
    payload = json.loads(pubsub_message)
    print(payload)

    return '200'

## **Billing**

In [ ]:
import base64
import base64
import json
import os
from googleapiclient import discovery

#PROJECT_ID = os.getenv('GCP_PROJECT')
PROJECT_ID = '<project ID>' #To be replaced 
PROJECT_NAME = f'projects/{PROJECT_ID}'


def stop_billing(data, context):
    pubsub_data = base64.b64decode(data['data']).decode('utf-8')
    pubsub_json = json.loads(pubsub_data)
    cost_amount = pubsub_json['costAmount']
    budget_amount = pubsub_json['budgetAmount']
    if cost_amount <= budget_amount:
        print(f'No action necessary. (Current cost: {cost_amount})')
        return

    if PROJECT_ID is None:
        print('No project specified with environment variable')
        return '200'

    billing = discovery.build(
        'cloudbilling',
        'v1',
        cache_discovery=False,
    )

    projects = billing.projects()

    billing_enabled = __is_billing_enabled(PROJECT_NAME, projects)

    if billing_enabled:
        __disable_billing_for_project(PROJECT_NAME, projects)
        print('billing was supposed to be disabled')
        return '200'
    else:
        print('Billing already disabled')
        return '200'


def __is_billing_enabled(project_name, projects):
    """
    Determine whether billing is enabled for a project
    @param {string} project_name Name of project to check if billing is enabled
    @return {bool} Whether project has billing enabled or not
    """
    try:
        res = projects.getBillingInfo(name=project_name).execute()
        return res['billingEnabled']
    except KeyError:
        # If billingEnabled isn't part of the return, billing is not enabled
        return False
    except Exception:
        print('Unable to determine if billing is enabled on specified project, assuming billing is enabled')
        return True


def __disable_billing_for_project(project_name, projects):
    """
    Disable billing for a project by removing its billing account
    @param {string} project_name Name of project disable billing on
    """
    body = {'billingAccountName': ''}  # Disable billing
    try:
        res = projects.updateBillingInfo(name=project_name, body=body).execute()
        print(f'Billing disabled: {json.dumps(res)}')
    except Exception:
        print('Failed to disable billing, possibly check permissions')